## Test set

### Script para encontrar um grupo de genes comuns em todas as redes para ser usado como test set


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mygene
import h5py
import networkx as nx
import seaborn as sns
%matplotlib inline


In [2]:
labels_unity = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final_v2/UNITY_labels_semisupervised.tsv', sep='\t')

labels_unity['label'].value_counts(dropna = False)

# 12918 passengers (False)
# 5777  candidates (NaN)
# 907   drivers    (True)

False    12918
NaN       5777
True       907
Name: label, dtype: int64

------------------------------------------

- A partir dos 19602 genes que tenho no total preciso criar um set com uma quantidade em comum em todas as redes
- O primeiro passo é comparar todas as redes e criar um dataframe concatenado que contenha todas
- Remover os genes que são diferentes e manter o número minimo levando em conta a menor rede (HPRD) devido a menor quantidade de genes

In [3]:
labels_hprd = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final_v2/HPRD_labels_semisupervised.tsv', sep='\t')

labels_multinet = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final_v2/MULTINET_labels_semisupervised.tsv', sep='\t')

labels_iref = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final_v2/IREF_labels_semisupervised.tsv', sep='\t')

labels_cpdb = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final_v2/CPDB_labels_semisupervised.tsv', sep='\t')

labels_pcnet = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final_v2/PCNET_labels_semisupervised.tsv', sep='\t')

labels_string = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final_v2/STRING_labels_semisupervised.tsv', sep='\t')

In [4]:
labels_unity['hprd'] = labels_unity['gene'].isin(labels_hprd['gene'])
labels_unity['multinet'] = labels_unity['gene'].isin(labels_multinet['gene'])
labels_unity['iref'] = labels_unity['gene'].isin(labels_iref['gene'])
labels_unity['cpdb'] = labels_unity['gene'].isin(labels_cpdb['gene'])
labels_unity['pcnet'] = labels_unity['gene'].isin(labels_pcnet['gene'])
labels_unity['string'] = labels_unity['gene'].isin(labels_string['gene'])

In [5]:
labels_unity

,gene,label,hprd,multinet,iref,cpdb,pcnet,string
0,A1BG,False,True,True,True,True,True,True
1,A1CF,True,True,True,True,True,True,True
2,A2M,False,True,True,True,True,True,True
3,A2ML1,NaN,False,False,False,True,True,True
4,A3GALT2,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...
19597,ZYG11A,False,False,False,True,False,True,True
19598,ZYG11B,False,True,True,True,True,True,True
19599,ZYX,False,True,True,True,True,True,True
19600,ZZEF1,NaN,True,True,True,True,True,True


In [6]:
labels_unity['string'].value_counts(dropna = False)

True     17872
False     1730
Name: string, dtype: int64

In [7]:
# verificando quantos genes tem dentro da rede unity de cada rede

# HPRD      9438
# MULTINET  13987
# IREF      14627
# CPDB      16243
# PCNET     19116
# STRING    17872

# Todas as redes estão completas dentro da Unity
# Próximo passo é retirar os genes em que todas as colunas são true (apenas as colunas das redes, não as dos labels)

# 11534 genes distintos, significa que 8068 são comum a todas as redes (VERIFICAR)

In [8]:
df1 = labels_unity[labels_unity['hprd'] == True]
df1

,gene,label,hprd,multinet,iref,cpdb,pcnet,string
0,A1BG,False,True,True,True,True,True,True
1,A1CF,True,True,True,True,True,True,True
2,A2M,False,True,True,True,True,True,True
14,AAGAB,False,True,True,True,True,True,True
17,AAMP,NaN,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...
19593,ZWINT,NaN,True,True,True,True,True,True
19596,ZXDC,False,True,True,True,True,True,True
19598,ZYG11B,False,True,True,True,True,True,True
19599,ZYX,False,True,True,True,True,True,True


In [9]:
df2 = df1[df1['multinet'] == True]
df3 = df2[df2['iref'] == True]
df4 = df3[df3['cpdb'] == True]
df5 = df4[df4['pcnet'] == True]
df6 = df5[df5['string'] == True]

In [10]:
df6['string'].value_counts(dropna = False)

True    8068
Name: string, dtype: int64

In [11]:
df6

,gene,label,hprd,multinet,iref,cpdb,pcnet,string
0,A1BG,False,True,True,True,True,True,True
1,A1CF,True,True,True,True,True,True,True
2,A2M,False,True,True,True,True,True,True
14,AAGAB,False,True,True,True,True,True,True
17,AAMP,NaN,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...
19593,ZWINT,NaN,True,True,True,True,True,True
19596,ZXDC,False,True,True,True,True,True,True
19598,ZYG11B,False,True,True,True,True,True,True
19599,ZYX,False,True,True,True,True,True,True


In [12]:
test_set = df6.drop(columns=['hprd', 'multinet', 'iref', 'cpdb', 'pcnet', 'string'])
test_set

,gene,label
0,A1BG,False
1,A1CF,True
2,A2M,False
14,AAGAB,False
17,AAMP,NaN
...,...,...
19593,ZWINT,NaN
19596,ZXDC,False
19598,ZYG11B,False
19599,ZYX,False


In [13]:
test_set['label'].value_counts(dropna = False)

False    4075
NaN      3261
True      732
Name: label, dtype: int64

In [14]:
test_set.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final_v2/test_set.tsv', sep='\t', index=False)

In [ ]:
test_set_labeled = test_set.dropna()
test_set_labeled

In [ ]:
positivos = test_set_labeled.loc[test_set_labeled['label'] == True]
negativos = test_set_labeled.loc[test_set_labeled['label'] == False]

positivos_sample = positivos.sample(n = 159)
negativos_sample = negativos.sample(n = 975)

testset2 = pd.concat([positivos_sample, negativos_sample])
testset2.sort_values('gene')

In [ ]:
testset2.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final_v2/test_set_final.tsv', sep='\t', index=False)